# Reconnaissance d'entités nommées avec SpaCy

La documentation est accessible ici: https://spacy.io/api

## Imports

In [5]:
from collections import defaultdict
import os
import spacy
from spacy.lang.fr.examples import sentences
!python -m spacy download fr_core_news_md

2025-11-03 21:23:56.530707: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-03 21:23:58.109719: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/-fr_core_news_md/-fr_core_news_md.tar.gz#egg===fr_core_news_md contains an egg fragment with a non-PEP 508 name. pip 25.3 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/13157

[notice] A new release of pi

In [6]:
nlp = spacy.load('fr_core_news_md')

### Reconnaissance des entités dans le corpus de 1965

In [ ]:
annee_choisie = "1965"
folder_path = r"C:\Users\tommy\TAC2\TAC\data\txt"
corpus_file = os.path.join(folder_path, f"corpus_{annee_choisie}.txt")



# Charger le texte complet de 1965
corpus_file = os.path.join(folder_path, f"corpus_{annee_choisie}.txt")
with open(corpus_file, 'r', encoding='utf-8') as f:
    texte = f.read()

# Analyse linguistique complète du corpus
nlp.max_length = 3_000_000  # ← augmente la limite à 3 millions de caractères
doc = nlp(texte)

# Découper le texte en phrases
sentences = list(doc.sents)

# Parcourir chaque phrase et extraire les entités
for sent in sentences:
    entities = [f"{ent.text} ({ent.label_})" for ent in sent.ents]
    if entities:
        print(f"→ '{sent.text.strip()}' contient les entités : {', '.join(entities)}")

→ '====
14 LE SOIR MERCREDI 6 JANVIER 1965' contient les entités : JANVIER 1965 (MISC)
→ 'n° 5812 sociétés BOURSES ETRANGERES' contient les entités : BOURSES ETRANGERES (MISC)
→ 'Belge 5 1/2' contient les entités : Belge 5 1/2 (MISC)
→ 'Belge 5 1/4 % 62-77 S ;' contient les entités : Belge (LOC), S (LOC)
→ 'Fonds Routes 5 3/4 % 60-75 $/DM : 105 1/2' contient les entités : Fonds Routes (MISC)
→ '- 106 1/2 : Idem DM/5 : 102 1/4' contient les entités : Idem DM/5 (PER)
→ 'Sabena 3/4 % 59-74 DM/S : 102 1/4 - 103 1/;» ;' contient les entités : Sabena (ORG), S (LOC)
→ 'R. T. T. 5 1/2 % 58-73' contient les entités : R. T. T. 5 1/2 (PER)
→ 'Ire et 2e série S : 102 A. ;' contient les entités : Ire (MISC), S (MISC)
→ 'S. N. C. B. 61-76 5 1/4 % $/DM : 99 7/8- 100 7/S ;' contient les entités : S. N. C. B. 61-76 (PER), S (LOC)
→ 'O. N. P. T. I. 5 1/2' contient les entités : O. N. P. T. I. 5 1/2 (PER)
→ '5.000 S : 99 3/4' contient les entités : S (LOC)
→ 'Petrofina 7 7 1/2' contient les entités : Pet

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [12]:
# Charger le texte
n=1000000
text = open("../data/all.txt", encoding='utf-8').read()[:n]

In [13]:
%%time
# Traiter le texte

doc = nlp(text)

CPU times: total: 23.6 s
Wall time: 24.2 s


In [15]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [16]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

Thérèse apparait 24 fois dans le corpus
Guilbert apparait 22 fois dans le corpus
M. Wilson apparait 20 fois dans le corpus
Rossel apparait 20 fois dans le corpus
Gioconda apparait 18 fois dans le corpus
Molière apparait 17 fois dans le corpus
Enguerrand apparait 17 fois dans le corpus
Wilson apparait 16 fois dans le corpus
Daubrac apparait 15 fois dans le corpus
Parentis apparait 13 fois dans le corpus
Savinien apparait 13 fois dans le corpus
Octave apparait 12 fois dans le corpus
Agence Rossel apparait 12 fois dans le corpus
Monsieur apparait 12 fois dans le corpus
Edwige apparait 12 fois dans le corpus
duc d’Aumale apparait 11 fois dans le corpus
jo no apparait 9 fois dans le corpus
Jules Ferry apparait 9 fois dans le corpus
Naissances apparait 9 fois dans le corpus
Reine apparait 9 fois dans le corpus
Monseigneur apparait 9 fois dans le corpus
Graux apparait 8 fois dans le corpus
Finet apparait 8 fois dans le corpus
Ali-Baba apparait 8 fois dans le corpus
Henri apparait 8 fois dans 

Exercice: essayez de lister les lieux (LOC) et les organisations (ORG) les plus mentionnées dans le corpus